### Read in impure dataset

In [201]:
import os
import pandas as pd
import re
import numpy as np
import string 
from nltk.tokenize import RegexpTokenizer
from nltk.tokenize import TweetTokenizer
from nltk.stem import PorterStemmer
import html.parser as html
import nltk
from nltk.corpus import stopwords 

from sklearn.feature_extraction.text import TfidfVectorizer 
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score 
from sklearn.model_selection import cross_val_predict



In [202]:

filename = 'processed_data.csv'
df = pd.read_csv(filename, encoding ='utf_8')


### Identifiying Tags we want to keep

In [203]:
df.head()

,text,user_id,in_reply_to_screen_name,followers_count,retweet_count,created_at,id_str,place_type,full_place_name,country_code,city,state,before_stemmitizing,final
0,@theblaze @realDonaldTrump https://t.co/TY9DlZ...,366636488,theblaze,186,0,Fri Aug 12 10:04:00 +0000 2016,764039724818272256,city,"Frontenac, MO",US,Frontenac,MO,"['@theblaze', '@realdonaldtrump']",@theblaz @realdonaldtrump
1,@BarackObama \n@FBI\n@LORETTALYNCH \nALL IN CO...,82496193,BarackObama,531,0,Fri Aug 12 10:04:02 +0000 2016,764039733076897792,city,"Baton Rouge, LA",US,Baton Rouge,LA,"['@barackobama', '@fbi', '@lorettalynch', 'col...",@barackobama @fbi @lorettalynch collus togeth ...
2,@theblaze @realDonaldTrump https://t.co/n050DB...,366636488,theblaze,186,0,Fri Aug 12 10:04:10 +0000 2016,764039769244348417,city,"Frontenac, MO",US,Frontenac,MO,"['@theblaze', '@realdonaldtrump']",@theblaz @realdonaldtrump
3,#CNN #newday clear #Trump deliberately throwin...,769208504,NaN,2652,0,Fri Aug 12 10:04:30 +0000 2016,764039849850482689,city,"Baltimore, MD",US,Baltimore,MD,"['#cnn', '#newday', 'clear', '#trump', 'delibe...",#cnn #newday clear #trump deliber throw race h...
4,"@realDonaldTrump, you wouldn't recognize a lie...",41043316,realDonaldTrump,183,0,Fri Aug 12 10:04:46 +0000 2016,764039917924069376,city,"Palm Springs, CA",US,Palm Springs,CA,"['@realdonaldtrump', 'recognize', 'lie', 'came...",@realdonaldtrump recogn lie came mouth continu...


### Geo_Tweets Dataframe

### Autolabel a training set


So my thought is that there are some very obvious tags that will tell us whether a tweet will be positive or negative towards a candidate. Chances are if someone tweets #imwithher the rest of the tweet is going to be pro hillary.

I will use a set of these tags to label tweets, then the classifier can look at the other words inside of it and find commonalities that it will later use to classify other tweets. 


The overall procedure is pretty much the same as in our werkcollege and lecture.

In [204]:
hillary_positive_tags = ['#imwithher', '#clintonkaine16', '#voteclinton', '#hillafornia', '#hilyes', '#voteblue']
hillary_negative_tags = ['#neverhillary', '#crookedhillary', '#nomoreclinton', '#hillaryshealth']
hillary_tags = ['hillary','hilary','Clinton','@hillaryclinton', '#hillaryclinton', '#hillary', '#clinton']+hillary_positive_tags+hillary_negative_tags

trump_positive_tags = ['#teamtrump','#trumppence16', '#maga', '#trumptrain','#womenfortrump']
trump_negative_tags = ['#dumptrump', '#nevertrump', '#conmanbaby', '#drumpf','#lyindonald']
trump_tags = ['@realdonaldtrump', '#trump', '#donaldtrump', 'trump','donald']+trump_negative_tags+trump_positive_tags


def has_trump_tag(row):
    
    text_in_lower_case = row['text'].lower()
    if any (word in text_in_lower_case for word in trump_positive_tags):
        return 'Positive'
    elif any (word in text_in_lower_case for word in trump_negative_tags):
        return 'Negative'


def has_hillary_tag(row):
    
    text_in_lower_case = row['text'].lower()
    if any (word in text_in_lower_case for word in hillary_positive_tags):
        return 'Positive'
    elif any (word in text_in_lower_case for word in hillary_negative_tags):
        return 'Negative'
    
    

def word_feats(words):
        return dict([(word, True) for word in words])


In [205]:
df['trump_sentiment_set']=df.apply(lambda row: has_trump_tag(row), axis=1)
df['hillary_sentiment_set']=df.apply(lambda row: has_hillary_tag(row), axis=1)


After taging the rows with unambiguous tweets I am splitting them into seperate dataframes

In [206]:
trump_pos= df[df['trump_sentiment_set']=='Positive']
trump_neg= df[df['trump_sentiment_set']=='Negative']

hillary_pos= df[df['hillary_sentiment_set']=='Positive']
hillary_neg= df[df['hillary_sentiment_set']=='Negative']



Now I go through all the words in the unambiguous tweets and assign them into dictionairies that can be used to train the classifier

In [207]:
# Find the features that most correspond to negative reviews    
trump_negfeats = [(word_feats(r['final'].split()), 'Negative') for i,r in trump_neg.iterrows()]

# Find the features that most correspond to positive reviews
trump_posfeats = [(word_feats(r['final'].split()), 'Positive') for i,r in trump_pos.iterrows()]


# Find the features that most correspond to negative reviews    
hillary_negfeats = [(word_feats(r['final'].split()), 'Negative') for i,r in hillary_neg.iterrows()]

# Find the features that most correspond to positive reviews
hillary_posfeats = [(word_feats(r['final'].split()), 'Positive') for i,r in hillary_pos.iterrows()]


In [208]:
trump_negcutoff = int(len(trump_negfeats)*9/10)
trump_poscutoff = int(len(trump_posfeats)*9/10)
hillary_negcutoff = int(len(hillary_negfeats)*9/10)
hillary_poscutoff = int(len(hillary_posfeats)*9/10)

print('Our sets are: Positive Training Set Hilary/Trump: %d / %d , Negative Trainig Set Hilary/Trump: %d / %d ' % (hillary_poscutoff , trump_poscutoff,hillary_negcutoff , trump_negcutoff))

Our sets are: Positive Training Set Hilary/Trump: 12978 / 20477 , Negative Trainig Set Hilary/Trump: 16151 / 15959 


### Lets start with Trump

In [209]:
trump_trainfeats = trump_negfeats[:trump_negcutoff] + trump_posfeats[:trump_poscutoff]
trump_testfeats = trump_negfeats[trump_negcutoff:] + trump_posfeats[trump_poscutoff:]


In [210]:
import nltk.classify.util
from nltk.classify import NaiveBayesClassifier

trump_classifier = NaiveBayesClassifier.train(trump_trainfeats)

In [211]:
print ('accuracy:', nltk.classify.util.accuracy(trump_classifier, trump_testfeats))
trump_classifier.show_most_informative_features(20)

accuracy: 0.9918518518518519
Most Informative Features
                    #fnc = True           Negati : Positi =    327.2 : 1.0
          #nomoreclinton = True           Positi : Negati =    231.0 : 1.0
           #americafirst = True           Positi : Negati =    217.1 : 1.0
            #sickhillari = True           Positi : Negati =    168.1 : 1.0
                    nick = True           Negati : Positi =    167.1 : 1.0
                #imwithy = True           Positi : Negati =    163.4 : 1.0
                @dbargen = True           Positi : Negati =    153.5 : 1.0
              #dumptrump = True           Negati : Positi =    145.1 : 1.0
         #hillaryshealth = True           Positi : Negati =    117.7 : 1.0
             #lyindonald = True           Negati : Positi =    112.5 : 1.0
           #mcmullin2016 = True           Negati : Positi =    108.0 : 1.0
                #voteblu = True           Negati : Positi =     95.9 : 1.0
         #crookedhillari = True           Pos

### Now Hillary

In [212]:
hillary_trainfeats = hillary_negfeats[:hillary_negcutoff] + hillary_posfeats[:hillary_poscutoff]
hillary_testfeats = hillary_negfeats[hillary_negcutoff:] + hillary_posfeats[hillary_poscutoff:]


In [213]:
hillary_classifier = NaiveBayesClassifier.train(hillary_trainfeats)

In [214]:
print ('accuracy:', nltk.classify.util.accuracy(hillary_classifier, hillary_testfeats))
hillary_classifier.show_most_informative_features()

accuracy: 0.9789993823347746
Most Informative Features
         #strongertogeth = True           Positi : Negati =    423.4 : 1.0
         #hillary4prison = True           Negati : Positi =    329.7 : 1.0
              #teamtrump = True           Negati : Positi =    178.0 : 1.0
            #sickhillari = True           Negati : Positi =    163.7 : 1.0
           #neverhillari = True           Negati : Positi =    116.9 : 1.0
              #donthecon = True           Positi : Negati =    111.6 : 1.0
                #demexit = True           Negati : Positi =    111.0 : 1.0
         #crookedhillari = True           Negati : Positi =     90.3 : 1.0
               #uniteblu = True           Positi : Negati =     78.9 : 1.0
                    imag = True           Positi : Negati =     67.1 : 1.0


### OK, we are getting somewhere. We have two classifiers. Now lets apply it to the rest of our dataframe

In [215]:
#Lets run a small test on a random line and see if we classify it well

print(df.iloc[10]['text'],trump_classifier.classify(word_feats(df.iloc[10]['final'].split())))

@CribBoss @WesSmith123 @realDonaldTrump why isn't his rape allegations getting more attention cause it's seeming more probable he did it Negative


The classifier calculates a probability for a tweet being both positive and negative. For tweets only mentioning the name he has a fairly high value that it could be both poisitive and negative. We want to label the tweet therefore not with the most likely result blindly, but only when the classifier is very sure in the result that its definetly one and not the other.

In [217]:

cutoff= 0.99

#this is arbitratry and can be changed later

def trump_sentiment_guesser(row):
    neg=trump_classifier.prob_classify(word_feats(row['final'].split())).prob('Negative')
    pos=trump_classifier.prob_classify(word_feats(row['final'].split())).prob('Positive')
    if pos - neg > cutoff:
        return 'Positive'
    elif neg - pos > cutoff:
        return 'Negative'
    else: return 'Neutral'
        
def hillary_sentiment_guesser(row):
    neg=hillary_classifier.prob_classify(word_feats(row['final'].split())).prob('Negative')
    pos=hillary_classifier.prob_classify(word_feats(row['final'].split())).prob('Positive')
    if pos - neg > cutoff:
        return 'Positive'
    elif neg - pos > cutoff:
        return 'Negative'
    else: return 'Neutral'   



In [218]:
df['trump_sentiment_guess']=df.apply(lambda row: trump_sentiment_guesser(row),axis=1)

In [219]:
df['hillary_sentiment_guess']=df.apply(lambda row: hillary_sentiment_guesser(row),axis=1)

In [220]:
# in case we only want to fill out non pre filled ones use df[df['hillary_sentiment_set'].isnull()]
df.head(20)

,text,user_id,in_reply_to_screen_name,followers_count,retweet_count,created_at,id_str,place_type,full_place_name,country_code,city,state,before_stemmitizing,final,trump_sentiment_set,hillary_sentiment_set,trump_sentiment_guess,hillary_sentiment_guess
0,@theblaze @realDonaldTrump https://t.co/TY9DlZ...,366636488,theblaze,186,0,Fri Aug 12 10:04:00 +0000 2016,764039724818272256,city,"Frontenac, MO",US,Frontenac,MO,"['@theblaze', '@realdonaldtrump']",@theblaz @realdonaldtrump,None,None,Neutral,Neutral
1,@BarackObama \n@FBI\n@LORETTALYNCH \nALL IN CO...,82496193,BarackObama,531,0,Fri Aug 12 10:04:02 +0000 2016,764039733076897792,city,"Baton Rouge, LA",US,Baton Rouge,LA,"['@barackobama', '@fbi', '@lorettalynch', 'col...",@barackobama @fbi @lorettalynch collus togeth ...,None,None,Positive,Negative
2,@theblaze @realDonaldTrump https://t.co/n050DB...,366636488,theblaze,186,0,Fri Aug 12 10:04:10 +0000 2016,764039769244348417,city,"Frontenac, MO",US,Frontenac,MO,"['@theblaze', '@realdonaldtrump']",@theblaz @realdonaldtrump,None,None,Neutral,Neutral
3,#CNN #newday clear #Trump deliberately throwin...,769208504,NaN,2652,0,Fri Aug 12 10:04:30 +0000 2016,764039849850482689,city,"Baltimore, MD",US,Baltimore,MD,"['#cnn', '#newday', 'clear', '#trump', 'delibe...",#cnn #newday clear #trump deliber throw race h...,None,None,Neutral,Neutral
4,"@realDonaldTrump, you wouldn't recognize a lie...",41043316,realDonaldTrump,183,0,Fri Aug 12 10:04:46 +0000 2016,764039917924069376,city,"Palm Springs, CA",US,Palm Springs,CA,"['@realdonaldtrump', 'recognize', 'lie', 'came...",@realdonaldtrump recogn lie came mouth continu...,Negative,None,Negative,Neutral
5,"""Kid, you know, suing someone? Thats the most ...",762090248159371264,NaN,159,0,Fri Aug 12 10:04:48 +0000 2016,764039926161604608,city,"Secaucus, NJ",US,Secaucus,NJ,"['kid', 'know', 'suing', 'someone', 'thats', '...",kid know su someon that beauti thing human cou...,None,None,Neutral,Neutral
6,@HillaryClinton you ARE the co-founder of ISIS...,85879639,HillaryClinton,68,0,Fri Aug 12 10:04:48 +0000 2016,764039928116240384,city,"Irving, TX",US,Irving,TX,"['@hillaryclinton', 'co-founder', 'isis', 'cro...",@hillaryclinton co-found isi crook evil lie wi...,None,None,Neutral,Neutral
7,@mike4193496 @realDonaldTrump I TOTALLY CONCUR...,1507953240,mike4193496,244,0,Fri Aug 12 10:04:53 +0000 2016,764039948567576576,city,"Chesapeake Beach, MD",US,Chesapeake Beach,MD,"['@mike4193496', '@realdonaldtrump', 'totally'...",@mike4193496 @realdonaldtrump total concur ele...,None,None,Neutral,Neutral
8,@realDonaldTrump @elsolarverde What issues? Yo...,1365441361,realDonaldTrump,289,0,Fri Aug 12 10:05:00 +0000 2016,764039975583227904,city,"Battle Creek, MI",US,Battle Creek,MI,"['@realdonaldtrump', '@elsolarverde', 'issues'...",@realdonaldtrump @elsolarverd issu idiot claim...,None,None,Neutral,Neutral
9,Can't stand @HillaryClinton or @realDonaldTrum...,229262835,NaN,37,0,Fri Aug 12 10:05:04 +0000 2016,764039994247819264,city,"Middletown, KY",US,Middletown,KY,"[""can't"", 'stand', '@hillaryclinton', '@realdo...",can't stand @hillaryclinton @realdonaldtrump t...,None,None,Negative,Neutral


In [221]:
print('Trump sentiment estimate:\n', df['trump_sentiment_guess'].value_counts(),'\n\nHillary sentiment estimate:\n', df['hillary_sentiment_guess'].value_counts())

Trump sentiment estimate:
 Neutral     322374
Positive     43542
Negative     29515
Name: trump_sentiment_guess, dtype: int64 

Hillary sentiment estimate:
 Neutral     327115
Positive     36042
Negative     32274
Name: hillary_sentiment_guess, dtype: int64


In [232]:
df_test=pd.read_csv('test_data.csv')

In [233]:
df_test['trump_sentiment']=df_test['trump_sentiment'].str.capitalize()
df_test['hilary_sentiment']=df_test['hilary_sentiment'].str.capitalize()

In [234]:
df_test.rename(columns={"hilary_sentiment": "hillary_sentiment","final": "stemmatized","stemmitized": "final"}, inplace=True)
df_test.head()

,text,in_reply_to_screen_name,followers_count,retweet_count,created_at,place_type,full_place_name,country_code,text_has_tag,City,State,stemmatized,final,to_label,general_sentiment,trump_sentiment,hillary_sentiment
0,@HillaryClinton be proud of your husband! http...,HillaryClinton,1750,0,Wed Aug 31 18:03:16 +0000 2016,city,"Fairfield, CT",US,True,Fairfield,CT,"['@hillaryclinton', 'proud', 'husband']",@hillaryclinton proud husband,True,positive,Neutral,Positive
1,@julieblue917 @cgetcho @mountain_goats @Vicent...,julieblue917,8,0,Wed Aug 31 19:53:19 +0000 2016,city,"Olathe, KS",US,True,Olathe,KS,"['@julieblue917', '@cgetcho', '@mountain_goats...",@julieblue917 @cgetcho @mountain_goat @vicente...,True,negative,Positive,Neutral
2,@MTP \nWhy do you call Trumps trip a Side Show...,MTP,66,0,Wed Aug 31 21:05:28 +0000 2016,city,"Riverside, CA",US,True,Riverside,CA,"['@mtp', 'call', 'trumps', 'trip', 'side', 'sh...",@mtp call trump trip side show can't listen in...,True,negative,Negative,Neutral
3,@greta @realDonaldTrump Mr.Trump did a great j...,greta,745,0,Wed Aug 31 21:52:24 +0000 2016,city,"Daphne, AL",US,True,Daphne,AL,"['@greta', '@realdonaldtrump', 'mr.trump', 'gr...",@greta @realdonaldtrump mr.trump great job mex...,True,positive,Positive,Neutral
4,@HillaryClinton she's choking right now becaus...,HillaryClinton,7,0,Wed Aug 31 23:32:21 +0000 2016,city,"Delray Beach, FL",US,True,Delray Beach,FL,"['@hillaryclinton', 'choking', 'right', 'go']",@hillaryclinton choke right go,True,negative,Neutral,Negative


In [235]:
# this is preparing some testing dictionairies from our manual data

trump_pos_test= df_test[df_test['trump_sentiment']=='Positive']
trump_neg_test= df_test[df_test['trump_sentiment']=='Negative']

hillary_pos_test= df_test[df_test['hillary_sentiment']=='Positive']
hillary_neg_test= df_test[df_test['hillary_sentiment']=='Negative']


# Find the features that most correspond to negative reviews    
trump_negfeats_test = [(word_feats(r['final'].split()), 'Negative') for i,r in trump_neg_test.iterrows()]

# Find the features that most correspond to positive reviews
trump_posfeats_test = [(word_feats(r['final'].split()), 'Positive') for i,r in trump_pos_test.iterrows()]


# Find the features that most correspond to negative reviews    
hillary_negfeats_test = [(word_feats(r['final'].split()), 'Negative') for i,r in hillary_neg_test.iterrows()]

# Find the features that most correspond to positive reviews
hillary_posfeats_test = [(word_feats(r['final'].split()), 'Positive') for i,r in hillary_pos_test.iterrows()]

trump_testfeats_test = trump_negfeats_test + trump_posfeats_test
hillary_testfeats_test = hillary_negfeats_test + hillary_posfeats_test



Time to test the classifier against our hand labeled data

In [236]:
#First Hillary
print ('accuracy:', nltk.classify.util.accuracy(hillary_classifier, hillary_testfeats_test))

accuracy: 0.7606837606837606


In [237]:
#And Trump
print ('accuracy:', nltk.classify.util.accuracy(trump_classifier, trump_testfeats_test))

accuracy: 0.7514792899408284


In [238]:
df.to_csv('Sentiment labeled data.csv')

In [239]:

df_test.head()

,text,in_reply_to_screen_name,followers_count,retweet_count,created_at,place_type,full_place_name,country_code,text_has_tag,City,State,stemmatized,final,to_label,general_sentiment,trump_sentiment,hillary_sentiment
0,@HillaryClinton be proud of your husband! http...,HillaryClinton,1750,0,Wed Aug 31 18:03:16 +0000 2016,city,"Fairfield, CT",US,True,Fairfield,CT,"['@hillaryclinton', 'proud', 'husband']",@hillaryclinton proud husband,True,positive,Neutral,Positive
1,@julieblue917 @cgetcho @mountain_goats @Vicent...,julieblue917,8,0,Wed Aug 31 19:53:19 +0000 2016,city,"Olathe, KS",US,True,Olathe,KS,"['@julieblue917', '@cgetcho', '@mountain_goats...",@julieblue917 @cgetcho @mountain_goat @vicente...,True,negative,Positive,Neutral
2,@MTP \nWhy do you call Trumps trip a Side Show...,MTP,66,0,Wed Aug 31 21:05:28 +0000 2016,city,"Riverside, CA",US,True,Riverside,CA,"['@mtp', 'call', 'trumps', 'trip', 'side', 'sh...",@mtp call trump trip side show can't listen in...,True,negative,Negative,Neutral
3,@greta @realDonaldTrump Mr.Trump did a great j...,greta,745,0,Wed Aug 31 21:52:24 +0000 2016,city,"Daphne, AL",US,True,Daphne,AL,"['@greta', '@realdonaldtrump', 'mr.trump', 'gr...",@greta @realdonaldtrump mr.trump great job mex...,True,positive,Positive,Neutral
4,@HillaryClinton she's choking right now becaus...,HillaryClinton,7,0,Wed Aug 31 23:32:21 +0000 2016,city,"Delray Beach, FL",US,True,Delray Beach,FL,"['@hillaryclinton', 'choking', 'right', 'go']",@hillaryclinton choke right go,True,negative,Neutral,Negative


In [240]:
#Quality testing, calculatingh precision, recall, f scores, accuracy
df_test['error_t_trump']=df_test.apply(lambda x: 0 if trump_sentiment_guesser(x) == x['trump_sentiment'] else (1 if x['trump_sentiment']== 'Neutral' else 2),axis=1)
df_test['error_t_hillary']=df_test.apply(lambda x: 0 if hillary_sentiment_guesser(x) == x['hillary_sentiment'] else (1 if x['hillary_sentiment']== 'Neutral' else 2),axis=1)

trump_recall= df_test['error_t_trump'].value_counts()[0]/((df_test['error_t_hillary'].value_counts()[0]+df_test['error_t_hillary'].value_counts()[2]))
hillary_recall= df_test['error_t_hillary'].value_counts()[0]/((df_test['error_t_hillary'].value_counts()[0]+df_test['error_t_hillary'].value_counts()[2]))
trump_precision=df_test['error_t_trump'].value_counts()[0]/((df_test['error_t_hillary'].value_counts()[0]+df_test['error_t_hillary'].value_counts()[1]))
hillary_precision=df_test['error_t_hillary'].value_counts()[0]/((df_test['error_t_hillary'].value_counts()[0]+df_test['error_t_hillary'].value_counts()[1]))
trump_f= 2* (trump_precision*trump_recall/(trump_precision+trump_recall))
hillary_f= 2* (hillary_precision*hillary_recall/(hillary_precision+hillary_recall))

trump_accuracy= df_test['error_t_trump'].value_counts()[0]/len(df_test.index)
hillary_accuracy= df_test['error_t_hillary'].value_counts()[0]/len(df_test.index)



print('Trump recall:',trump_recall,'\n', 
      'Clinton recall:',hillary_recall,'\n','\n',
     'Trump precision:',trump_precision,'\n', 
      'Clinton precision:',hillary_precision,'\n','\n',
     'Trump f-score:',trump_f,'\n', 
      'Clinton f-score:',hillary_f,'\n' ,'\n',
     'Trump accuracy:',trump_accuracy,'\n', 
      'Clinton accuracy:',hillary_accuracy)



Trump recall: 0.5416666666666666 
 Clinton recall: 0.6893939393939394 
 
 Trump precision: 0.7009803921568627 
 Clinton precision: 0.8921568627450981 
 
 Trump f-score: 0.6111111111111109 
 Clinton f-score: 0.7777777777777779 
 
 Trump accuracy: 0.5 
 Clinton accuracy: 0.6363636363636364


In [ ]:
 df.groupby(['trump_sentiment_guess','hillary_sentiment_guess']).size()


,text,in_reply_to_screen_name,followers_count,retweet_count,created_at,place_type,full_place_name,country_code,text_has_tag,City,State,final,stemmitized,to_label,general_sentiment,trump_sentiment,hillary_sentiment
0,@HillaryClinton be proud of your husband! http...,HillaryClinton,1750,0,Wed Aug 31 18:03:16 +0000 2016,city,"Fairfield, CT",US,True,Fairfield,CT,"['@hillaryclinton', 'proud', 'husband']",@hillaryclinton proud husband,True,positive,Neutral,Positive
1,@julieblue917 @cgetcho @mountain_goats @Vicent...,julieblue917,8,0,Wed Aug 31 19:53:19 +0000 2016,city,"Olathe, KS",US,True,Olathe,KS,"['@julieblue917', '@cgetcho', '@mountain_goats...",@julieblue917 @cgetcho @mountain_goat @vicente...,True,negative,Positive,Neutral
2,@MTP \nWhy do you call Trumps trip a Side Show...,MTP,66,0,Wed Aug 31 21:05:28 +0000 2016,city,"Riverside, CA",US,True,Riverside,CA,"['@mtp', 'call', 'trumps', 'trip', 'side', 'sh...",@mtp call trump trip side show can't listen in...,True,negative,Negative,Neutral
3,@greta @realDonaldTrump Mr.Trump did a great j...,greta,745,0,Wed Aug 31 21:52:24 +0000 2016,city,"Daphne, AL",US,True,Daphne,AL,"['@greta', '@realdonaldtrump', 'mr.trump', 'gr...",@greta @realdonaldtrump mr.trump great job mex...,True,positive,Positive,Neutral
4,@HillaryClinton she's choking right now becaus...,HillaryClinton,7,0,Wed Aug 31 23:32:21 +0000 2016,city,"Delray Beach, FL",US,True,Delray Beach,FL,"['@hillaryclinton', 'choking', 'right', 'go']",@hillaryclinton choke right go,True,negative,Neutral,Negative
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
281,@HillaryClinton you are funded by the top 1%. ...,HillaryClinton,37,0,Thu Aug 18 16:30:33 +0000 2016,city,"Monterey, CA",US,True,Monterey,CA,"['@hillaryclinton', 'funded', 'top', 'honestly...",@hillaryclinton fund top honestli trust,True,positive,Neutral,Negative
282,@realDonaldTrump The best thing for USA is tha...,realDonaldTrump,93450,0,Thu Aug 18 16:35:34 +0000 2016,city,"Clinton, MD",US,True,Clinton,MD,"['@realdonaldtrump', 'best', 'thing', 'usa', '...",@realdonaldtrump best thing usa #nevertrump #l...,True,positive,Negative,Positive
283,@realDonaldTrump Do you feel superior to peopl...,realDonaldTrump,46,0,Thu Aug 18 16:41:18 +0000 2016,city,"Toronto, OH",US,True,Toronto,OH,"['@realdonaldtrump', 'feel', 'superior', 'peop...",@realdonaldtrump feel superior peopl rich thin...,True,negative,Negative,Neutral
284,@foxnewspolitics @realDonaldTrump Nice Try #Cr...,foxnewspolitics,5242,0,Thu Aug 18 16:46:20 +0000 2016,city,"Los Angeles, CA",US,True,Los Angeles,CA,"['@foxnewspolitics', '@realdonaldtrump', 'nice...",@foxnewspolit @realdonaldtrump nice tri #crook...,True,neutral,Neutral,Negative
